In [23]:
import sys
import numpy as np
import import_ipynb
from pathlib import Path
from scipy.sparse.linalg import spsolve

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from pre_processing import create_domain, read_mesh
    from fem_processing import boundary_conditions,matrices_assembly
    from pos_processing import graph_results
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Modules imports were successful!


# Project 1: Coaxial Cable

Considere um cabo coaxial com dois dielétricos apresentado na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/coaxial_problem.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of Coaxial Problem.</figcaption>
</figure>

O raio do condutor interno é ``a`` e do condutor externo é ``b``. O primeiro dielétrico, $\varepsilon_{1}$, preenche o anel circular entre os raios ``a`` e ``c``, enquanto o segundo dielétrico, $\varepsilon_{2}$, preenche o anel entre os raios ``c`` e ``b``.  

Entre o condutor interno e o externo é aplicada uma diferença de potencial igual a $V$.  

Seja $V = 1$ (potencial zero em ``a`` e um em ``b``), $a = 2 \, mm$, $b = 8 \, mm$, $c = 5 \, mm$, $\varepsilon_{r1} = 2$ e $\varepsilon_{r2} = 4$. Para este problema, além da convergência nas normas $L_2$ e da energia, calcule a convergência para o valor da capacitância do dispositivo.

# Bilinear Quadrilateral Lagrangian Element $Q_1$
## Pre-processor module
### Geometry and mesh Domain

In [24]:
FINITE_ELEMENT = ("Quadrangle", 1)

BOUNDARY = [{'type': 'Dirichlet', 'tag': 101, 'value': 0.0, 'name': 'inner_conductor'},
                       {'type': 'Dirichlet', 'tag': 102, 'value': 1.0, 'name': 'outer_conductor'}]

MATERIAL = [{'type': 'relative_electric_permittivity', 'tag': 201, 'value': 2, 'name': 'medium_1'},
                        {'type': 'relative_electric_permittivity', 'tag': 202, 'value': 4, 'name': 'medium_2'}]

# Create mesh from file geometry
create_domain.coaxial_geometry(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=0.5E-3, view_mesh=True)
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY, MATERIAL, model='coaxial')
graph_results.plot_mesh(mesh_data, FINITE_ELEMENT, model='coaxial')

Model coaxial_domain_Quadrangle1 (2D)
Info     : 8 geometric entities
Info     : 4 Physical Groups
Info     : 874 nodes in total
Info     : 810 2-D elements in total
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pre_processing\pictures\coaxial_meshed_domain_Quadrangle1.svg


### Mesh Data Dictionaries
#### Node Data

In [25]:
mesh_nodes = mesh_data['nodes_data']; mesh_nodes[:1]

[{'TagID': 1,
  'global_coord': (0.002, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}}]

#### Node List

In [26]:
nodes = [node['global_coord'] for node in mesh_data['nodes_data']]; nodes[:1]

[(0.002, 0.0)]

#### Conn Data

In [27]:
conn_data = mesh_data['conn_data']; conn_data[:1]

[{'TagID': 196,
  'conn_list': [436, 316, 378, 413],
  'material': {'type': 'relative_electric_permittivity',
   'tag': 201,
   'value': 2,
   'name': 'medium_1'}}]

#### Conn List

In [28]:
conn = [element['conn_list'] for element in mesh_data['conn_data']]; conn[:1]

[[436, 316, 378, 413]]

### Figure 1: Coaxial meshed domain 

In [29]:
graph_results.plot_mesh(mesh_data, FINITE_ELEMENT, model='coaxial', Numbering=False)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pre_processing\pictures\coaxial_meshed_domain_Quadrangle1.svg


<figure>
    <img src="pre_processing/pictures/coaxial_meshed_domain_Quadrangle1.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Coaxial meshed domain.</figcaption>
</figure>

### Assembly Matrices
#### `assembly.global_nodes_coordinates()` 

In [30]:
ai, xi, yi = matrices_assembly.global_nodes_coordinates(0, mesh_data)
print(f"'a_1' Global coordinates: {ai}")
print(f"'x_1' x-coordinates: {xi}")
print(f"'y_1' y-coordinates: {yi}")

'a_1' Global coordinates: [(0.001675337138027499, 0.002991401944730367), (0.00192559069720858, 0.003558996471404466), (0.001292637235637424, 0.003711305422351227), (0.001174143592839745, 0.003184936830131519)]
'x_1' x-coordinates: [0.001675337138027499, 0.00192559069720858, 0.001292637235637424, 0.001174143592839745]
'y_1' y-coordinates: [0.002991401944730367, 0.003558996471404466, 0.003711305422351227, 0.003184936830131519]


In [31]:
#### `map_to_physical_coordinates()`

In [32]:
xi_master = (1, 0)
xg, yg = matrices_assembly.isomapping_to_global_coordinates(ai, xi_master, FINITE_ELEMENT)
print(f"'e_1' Master coordinates: {xi_master} --> Global coordinates: ({xg}, {yg})")

'e_1' Master coordinates: (1, 0) --> Global coordinates: (0.001609113966423002, 0.0036351509468778463)


#### `apply_physics`

In [33]:
mesh_data = ps.apply_physics(mesh_data, FINITE_ELEMENT)
mesh_data['conn_data'][:1]

[{'TagID': 196,
  'conn_list': [436, 316, 378, 413],
  'material': {'type': 'relative_electric_permittivity',
   'tag': 201,
   'value': 2,
   'name': 'medium_1',
   'a_value': 1.77083756256e-11},
  'source': {'type': 'free_source', 'value': 0}}]

#### Material Propriety, $a$

In [34]:
a = [element['material']['a_value'] for element in mesh_data['conn_data']]
print("Material 'a' values: \n", np.array(a))

Material 'a' values: 
 [1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 1.77083756e-11 1.77083756e-11 1.77083756e-11 1.77083756e-11
 

#### Source: Analytical source, $f(x)$

In [35]:
Fx = [element['source']['value'] for element in mesh_data['conn_data']]
print("Source vector Fx: \n", np.array(Fx))

Source vector Fx: 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 

#### Linear Jacobian

In [36]:
Je = matrices_assembly.jacobian(e=0, mesh_data=mesh_data, element_type=FINITE_ELEMENT, xik=(0, 0)) 
Jdet, Jinv = np.linalg.det(Je), np.linalg.inv(Je)
print("Jacobian matrix for element e_0:\n", Je)
print("Determinant of the Jacobian matrix for element e_0:", Jdet)

Jacobian matrix for element e_0:
 [[ 9.21868005e-05  2.73490780e-04]
 [-2.83536752e-04  8.64609591e-05]]
Determinant of the Jacobian matrix for element e_0: 8.551524648580932e-08


#### Local Elements $e_1$

In [37]:
Ae, Fe, areae = matrices_assembly.higher_order_local_matrices(e=0, mesh_data=mesh_data, element_type=FINITE_ELEMENT)
print("Local stiffness matrix for element e_1:\n", Ae)
print("Local load vector for element e_1:\n", Fe)
print("Area of element e_1: ", areae)

Local stiffness matrix for element e_1:
 [[ 1.22523598e-11 -2.79482660e-12 -6.14923260e-12 -3.30830060e-12]
 [-2.79482660e-12  1.06916032e-11 -2.18800460e-12 -5.70877203e-12]
 [-6.14923260e-12 -2.18800460e-12  1.19342360e-11 -3.59699876e-12]
 [-3.30830060e-12 -5.70877203e-12 -3.59699876e-12  1.26140714e-11]]
Local load vector for element e_1:
 [[0.]
 [0.]
 [0.]
 [0.]]
Area of element e_1:  3.4206098594323663e-07


#### Global “stiffness” matrix, $A_g$

In [38]:
Ag, fg = matrices_assembly.get_global_matrix(mesh_data, FINITE_ELEMENT)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (874, 874)


### Imposition of Boundary Conditions

In [39]:
dirichlet_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
dirichlet_values = [
    node['bc']['value'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
free_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Free']

print(f"The entire domain has {len(nodes)} nodes: " 
      f"{len(nodes) - len(dirichlet_nodes)} free nodes and {len(dirichlet_nodes)} Dirichlet nodes.")
print("Free nodes: \n", np.array(free_nodes))
print("Dirichlet nodes: \n", np.array(dirichlet_nodes))
print("Dirichlet nodes values: \n", np.array(dirichlet_values))

The entire domain has 874 nodes: 746 free nodes and 128 Dirichlet nodes.
Free nodes: 
 [  2  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63
  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  90  91 193 194 195 196 197 198 199 200
 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236
 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254
 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272
 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290
 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308
 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326
 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344
 345 346 347 348 349 350 351 352 353 

### Mapping Global nodes to reduced system

This code creates a dictionary called ``global_to_reduced``, which maps the global indices of the mesh nodes to the reduced indices, i.e. the indices that correspond only to the nodes that are not in the _Dirichlet boundary conditions_.

In [40]:
global_to_reduced = {node: i for i, node in enumerate(
        idx_n for idx_n in range(1, len(nodes)+1) if idx_n not in dirichlet_nodes
    )}
print("Global to reduced nodes mapping:"); global_to_reduced

Global to reduced nodes mapping:


{2: 0,
 29: 1,
 30: 2,
 31: 3,
 32: 4,
 33: 5,
 34: 6,
 35: 7,
 36: 8,
 37: 9,
 38: 10,
 39: 11,
 40: 12,
 41: 13,
 42: 14,
 43: 15,
 44: 16,
 45: 17,
 46: 18,
 47: 19,
 48: 20,
 49: 21,
 50: 22,
 51: 23,
 52: 24,
 53: 25,
 54: 26,
 55: 27,
 56: 28,
 57: 29,
 58: 30,
 59: 31,
 60: 32,
 61: 33,
 62: 34,
 63: 35,
 64: 36,
 65: 37,
 66: 38,
 67: 39,
 68: 40,
 69: 41,
 70: 42,
 71: 43,
 72: 44,
 73: 45,
 74: 46,
 75: 47,
 76: 48,
 77: 49,
 78: 50,
 79: 51,
 80: 52,
 81: 53,
 82: 54,
 83: 55,
 84: 56,
 85: 57,
 86: 58,
 87: 59,
 88: 60,
 89: 61,
 90: 62,
 91: 63,
 193: 64,
 194: 65,
 195: 66,
 196: 67,
 197: 68,
 198: 69,
 199: 70,
 200: 71,
 201: 72,
 202: 73,
 203: 74,
 204: 75,
 205: 76,
 206: 77,
 207: 78,
 208: 79,
 209: 80,
 210: 81,
 211: 82,
 212: 83,
 213: 84,
 214: 85,
 215: 86,
 216: 87,
 217: 88,
 218: 89,
 219: 90,
 220: 91,
 221: 92,
 222: 93,
 223: 94,
 224: 95,
 225: 96,
 226: 97,
 227: 98,
 228: 99,
 229: 100,
 230: 101,
 231: 102,
 232: 103,
 233: 104,
 234: 105,
 235: 106

## Processor Module
### Reduced Global matrix $A_{gr}$ with boundary conditions

In [41]:
Agr, fgr = boundary_conditions.get_reduced_global_matrix(mesh_data, FINITE_ELEMENT)
print("Reduced global matrix shape:", Agr.shape)
print("Reduced global matrix: \n", Agr)
print("Reduced force vector: \n", fgr)

Reduced global matrix shape: (746, 746)
Reduced global matrix: 
   (0, 0)	7.166769876594893e-11
  (0, 1)	-8.449443961654953e-12
  (0, 63)	-8.538404691566495e-12
  (0, 78)	-5.074682088501574e-12
  (0, 102)	-6.5023230338905375e-12
  (0, 176)	-4.205246442817285e-12
  (0, 368)	-1.5118120348580006e-11
  (0, 451)	-1.1082586151662608e-11
  (0, 454)	-1.2696892047275462e-11
  (1, 0)	-8.449443961654953e-12
  (1, 1)	7.170387313861297e-11
  (1, 2)	-8.41413190573857e-12
  (1, 78)	-4.32310228293185e-12
  (1, 158)	-4.806239886367206e-12
  (1, 176)	-6.849873761314701e-12
  (1, 314)	-1.0875036655245769e-11
  (1, 368)	-1.2701413388315703e-11
  (1, 451)	-1.528463129704422e-11
  (2, 1)	-8.41413190573857e-12
  (2, 2)	7.18567219635964e-11
  (2, 3)	-8.041419656786926e-12
  (2, 78)	-7.153928560174213e-12
  (2, 103)	-4.526600285504155e-12
  (2, 158)	-4.58680938719087e-12
  (2, 314)	-1.538352030150579e-11
  (2, 451)	-1.2911506442406727e-11
  (2, 453)	-1.0838805424289149e-11
  (3, 2)	-8.041419656786926e-12
  (3,

### Global Potential Vector

In [42]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.get_global_potentials(mesh_data, ur)
print("The reduced solution vector ur is: \n", ur)
print("The global potential vector V is: \n", potential_u)

The reduced solution vector ur is: 
 [0.79591916 0.79593062 0.79594438 0.79593697 0.7958161  0.79557918
 0.79610027 0.79602866 0.7960003  0.79609203 0.79625236 0.79571344
 0.79599466 0.79616105 0.79623988 0.79626281 0.79643039 0.79607272
 0.79590239 0.79572325 0.79586401 0.79570922 0.79563712 0.79571072
 0.79650716 0.79578305 0.7954219  0.79589916 0.79605824 0.79626913
 0.79627782 0.79607409 0.79570417 0.79626407 0.79618954 0.79610202
 0.79606955 0.79596906 0.79599811 0.79616762 0.79575701 0.79590931
 0.79590307 0.79579113 0.79581522 0.7958651  0.79605768 0.79587815
 0.79567212 0.79648545 0.79624389 0.79615427 0.79614307 0.79614077
 0.79615377 0.79576553 0.79543368 0.79570361 0.79577083 0.79599422
 0.79629772 0.79608742 0.79596066 0.79591299 0.17833487 0.17678303
 0.16120924 0.16649023 0.24747593 0.18072579 0.20968818 0.19020222
 0.19828893 0.20629092 0.70360074 0.69044965 0.7001231  0.70691858
 0.70131067 0.73531485 0.69544625 0.69345201 0.70640631 0.69914596
 0.72759087 0.70207245 0.

### Solution at each element

In [43]:
u_list = []
for e in range(len(conn)):
    uh_values = [potential_u[id - 1] for id in conn[e]]
    u_list.append(uh_values)
print("The solution u for Node Element is: \n", np.array(u_list))

The solution u for Node Element is: 
 [[0.46957921 0.61221922 0.58697655 0.46027715]
 [0.37056015 0.50241796 0.50672116 0.40899886]
 [0.49131892 0.60751199 0.63126606 0.50997479]
 ...
 [0.90680278 0.90738184 0.87244021 0.87169253]
 [0.91000639 0.91592012 0.8847617  0.87519999]
 [0.94101968 0.9394207  0.90649552 0.90859569]]


## Post-Processor

In [44]:
graph_results.approximate_not_structured_quadrangles(FINITE_ELEMENT, mesh_data, potential_u, model='coaxial')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pos_processing\pictures\coaxial_domain_solution_Quadrangle1_not_structured.svg


### Figure 2: Scalar Field Distribution

<figure>
    <img src="pos_processing/pictures/coaxial_domain_solution_Quadrangle1_not_structured.svg" alt="Fig.2" style="width:100%;" />
    <figcaption>Figure 2: Post-Processor: Scalar Field Distribution.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.